In [12]:
import pandas as pd
import numpy as np
from selenium import webdriver
import json, pickle
import time

In [13]:
with open('champion_id_and_name_list.pickle', 'rb') as f:
    champ_dict = pickle.load(f)

In [15]:
champ_dict
champ_dict_id = list(champ_dict.keys())
champ_dict_name = list(champ_dict.values())

In [17]:
driver = webdriver.Chrome()
#언어를 영어로 셋팅
driver.get('https://op.gg/')
driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
time.sleep(5)
lang_list = driver.find_elements_by_class_name('setting-list__item')
for lang in lang_list:
    if "English" in lang.text:
        lang.click()
time.sleep(3)
driver.find_elements_by_class_name('setting__button')[0].click()
time.sleep(3)

driver.get('https://www.op.gg/champion/statistics')


In [26]:
lis = driver.find_elements_by_class_name('champion-index-trend-position__item')
print(len(lis))
for li in lis:
    time.sleep(2)
    li_a = li.find_element_by_xpath('./a')
    li_a.click()
    
pos_lis = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']
pos_frame = []
for pos in pos_lis:
    df = pd.DataFrame(columns = ['rank', 'champname', 'champid', 'winrate', 'pickrate', 'tier'])
    tbody = driver.find_element_by_css_selector(f'tbody.tabItem.champion-trend-tier-{pos}')
    trs = tbody.find_elements_by_xpath('./tr')
    for tr in trs:
        champ_data = {} #빈 프레임
        try:
            champ_data['rank'] = tr.find_element_by_css_selector('td.champion-index-table__cell--rank').get_attribute('innerText')
            champ_data['champname'] = tr.find_element_by_css_selector('td>a>div.champion-index-table__name').get_attribute('innerText')
            if champ_data['champname'] in champ_dict_name:
                champ_data['champid'] = champ_dict_id[champ_dict_name.index(champ_data['champname'])]
            else: champ_data['champid'] = 0
            champ_data['winrate'] = tr.find_element_by_xpath('./td[5]').get_attribute('innerText')
            champ_data['pickrate'] = tr.find_element_by_xpath('./td[6]').get_attribute('innerText')
            champ_data['tier'] = int(tr.find_element_by_xpath('./td[7]/img').get_attribute('src')[-5])
        except:
            pass
            
        if len(champ_data)>0:
            df = df.append(champ_data,  ignore_index=True)
    
    print(pos,'data num:',len(df))
    pos_frame.append(df)
    

5
TOP data num: 50
JUNGLE data num: 39
MID data num: 52
ADC data num: 23
SUPPORT data num: 39


In [31]:
pos_frame[0]

,rank,champname,champid,winrate,pickrate,tier
0,1,Malphite,54,51.28%,14.26%,1
1,2,Shen,98,54.88%,10.37%,1
2,3,Ornn,516,52.56%,13.09%,1
3,4,Camille,164,51.78%,12.49%,1
4,5,Darius,122,52.61%,4.53%,1
5,6,Maokai,57,52.37%,7.53%,2
6,7,Kayle,10,54.00%,5.11%,2
7,8,Aatrox,266,50.36%,8.38%,2
8,9,Akali,84,46.69%,6.48%,2
9,10,Volibear,106,50.14%,6.95%,2


In [30]:
pos_frame[1]

,rank,champname,champid,winrate,pickrate,tier
0,1,Amumu,32,54.85%,17.98%,1
1,2,Kayn,141,55.02%,20.43%,1
2,3,Graves,104,50.82%,20.97%,1
3,4,Nunu & Willump,20,52.98%,10.04%,2
4,5,Kha'Zix,121,50.24%,14.34%,2
5,6,Hecarim,120,50.66%,12.83%,2
6,7,Rammus,33,53.72%,4.27%,2
7,8,Master Yi,11,49.72%,5.68%,2
8,9,Rek'Sai,421,49.13%,4.13%,2
9,10,Zac,154,48.00%,6.01%,2


In [32]:
#pos_frame -> 5개 리스트
# 0 -top, 1 - jungle, 2- mid, 3-ad_carry, 4- support
with open('LOL_Champ_Tier.pickle', 'wb') as f:
    pickle.dump(pos_frame, f)